<a href="https://colab.research.google.com/github/SumeetsRoorkee/ML_Code/blob/main/Sentiment_analysis_colab_GPU_Vesrion_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from tqdm import tqdm
from textblob import TextBlob
import time
import re
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
tqdm.pandas()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data.csv')

In [6]:
df.reset_index(inplace = True, drop = True)

In [8]:
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [9]:
print(df.iloc[0])

Sentence     The GeoSolutions technology will leverage Bene...
Sentiment                                             positive
Name: 0, dtype: object


In [10]:
possible_labels = df.Sentiment.unique()

In [11]:
labels = {}
for index, possible_label in enumerate(possible_labels):
    labels[possible_label] = index

In [12]:
labels

{'positive': 0, 'negative': 1, 'neutral': 2}

In [13]:
df['labels'] = df.Sentiment.replace(labels)

In [14]:
df.drop('Sentiment', axis=1, inplace=True)

In [15]:
df.head()

,Sentence,labels
0,The GeoSolutions technology will leverage Bene...,0
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",1
2,"For the last quarter of 2010 , Componenta 's n...",0
3,According to the Finnish-Russian Chamber of Co...,2
4,The Swedish buyout firm has sold its remaining...,2


In [16]:
! pip install -q transformers
from transformers import pipeline

In [17]:
import torch
torch.cuda.is_available()

False

In [18]:
def preprocess(text):
    new_text = []


    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [20]:
df['Sentence'] = df['Sentence'].apply(preprocess)

In [21]:
!pip install datasets transformers huggingface_hub

In [22]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [23]:
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)

In [24]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

In [25]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(X_train)
test_dataset = Dataset.from_pandas(X_test)

In [26]:
X_train.head()

,Sentence,labels
0,The floor area of the Yliopistonrinne project ...,2
1,"no compensation for its news , opinions or dis...",2
2,RT @user $AAPL still on track for $500,0
3,This includes a EUR 39.5 mn change in the fair...,2
4,$GTE LONG at 7.44,0


In [28]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

In [29]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [30]:
def preprocess_function(examples):
    return tokenizer(examples['Sentence'])


In [31]:
tokenized_train = train_dataset.map(preprocess_function)
tokenized_test = test_dataset.map(preprocess_function)

Map:   0%|          | 0/4673 [00:00<?, ? examples/s]

Map:   0%|          | 0/1169 [00:00<?, ? examples/s]

In [32]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [33]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [35]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [36]:
from huggingface_hub import notebook_login
notebook_login()

In [37]:
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-model-3000-samples"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)



In [38]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


In [39]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sumeetsroorkee (sumeetsroorkee-everabank) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.535300


Step,Training Loss
500,0.535300


TrainOutput(global_step=586, training_loss=0.5016363912068129, metrics={'train_runtime': 1040.6351, 'train_samples_per_second': 8.981, 'train_steps_per_second': 0.563, 'total_flos': 275714597642274.0, 'train_loss': 0.5016363912068129, 'epoch': 2.0})

In [40]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/RoorkeeSumeet/finetuning-sentiment-model-3000-samples/commit/30f5d295efab7e519f9a89125e1e45c12609d183', commit_message='End of training', commit_description='', oid='30f5d295efab7e519f9a89125e1e45c12609d183', pr_url=None, repo_url=RepoUrl('https://huggingface.co/RoorkeeSumeet/finetuning-sentiment-model-3000-samples', endpoint='https://huggingface.co', repo_type='model', repo_id='RoorkeeSumeet/finetuning-sentiment-model-3000-samples'), pr_revision=None, pr_num=None)

In [41]:
from transformers import pipeline

sentiment_model = pipeline(model="federicopascual/finetuning-sentiment-model-3000-samples")
sentiment_model(['positive', 'negative', 'neutral'])


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'LABEL_0', 'score': 0.5432146787643433},
 {'label': 'LABEL_0', 'score': 0.7404693365097046},
 {'label': 'LABEL_0', 'score': 0.6129472255706787}]